In [14]:
import time
from tqdm import tqdm
import torch
import numpy as np
from torchani.datasets import COMP6v1
import torchani
from torchani.units import hartree2kcalmol
from torchani.utils import species_to_formula, ATOMIC_NUMBERS, PERIODIC_TABLE

In [4]:
ds = COMP6v1(root='/home/nick/Datasets/comp6v1_B973c/', basis_set='def2mTZVP', functional='B973c')
device = torch.device('cuda')
ani = torchani.models.ANIdr().to(device)

Checking integrity of files for dataset COMP6v1:   0%|          | 0/1 [00:00<?, ?it/s]

Name: COMP6-v1_full-B973c-def2mTZVP
ANI h5py store:
{'Conformer groups': 97,
 'Conformers': '99,968',
 'Properties': ['D3.energy-corrections',
                'D3.force-corrections',
                '_id',
                'coordinates',
                'dipole',
                'energies',
                'forces',
                'quadrupole',
                'species'],
 'Store Metadata': {}}


In [16]:
species = torch.tensor([[8, 1, 1, 8, 1, 1],
        [8, 1, 1, 8, 1, 1],
        [8, 1, 1, 8, 1, 1],
        [8, 1, 1, 8, 1, 1],
        [8, 1, 1, 8, 1, 1],
        [8, 1, 1, 8, 1, 1],
        [8, 1, 1, 8, 1, 1],
        [8, 1, 1, 8, 1, 1]])
atomic_symbols = [PERIODIC_TABLE[number] for row in species for number in row]
atomic_symbols_list = [atomic_symbols[i] for i in range(0, len(atomic_symbols), species.size(1))]


atomic_symbols_list = [[PERIODIC_TABLE[number] for number in row] for row in species]
print(atomic_symbols_list)

[['O', 'H', 'H', 'O', 'H', 'H'], ['O', 'H', 'H', 'O', 'H', 'H'], ['O', 'H', 'H', 'O', 'H', 'H'], ['O', 'H', 'H', 'O', 'H', 'H'], ['O', 'H', 'H', 'O', 'H', 'H'], ['O', 'H', 'H', 'O', 'H', 'H'], ['O', 'H', 'H', 'O', 'H', 'H'], ['O', 'H', 'H', 'O', 'H', 'H']]


In [25]:
_start = time.perf_counter()
atomic_symbols_list = []
with ds.keep_open("r") as read_ds:
    pbar = tqdm(total=ds.num_conformers)
    for group, j, conformer in read_ds.chunked_items(max_size=2500):
        species = conformer["species"].to(device)
        atomic_symbols_list.extend([PERIODIC_TABLE[number] for number in row] for row in species)
        print(atomic_symbols_list)
        break
        print(len(atomic_symbols_list))
        #print(species)
        coordinates = conformer["coordinates"].to(device)
        coordinates.requires_grad_(True)
        #print(coordinates.shape)
        energies = ani((species, coordinates)).energies
        #print(energies.shape)
        #print(energies.sum().shape)
        forces = -torch.autograd.grad(energies.sum(), coordinates)[0]
        #print(forces.shape)
        pbar.update(len(coordinates))
        print(pbar)
    pbar.close()
print(f"Time elapsed: {time.perf_counter() - _start} s")
# This took about 7.3 min in CPU of my laptop
# Intel(R) Core(TM) i5-10300H CPU @ 2.50GHz
# My laptop's GPU doesn't have enough memory to do it

  0%|          | 0/99968 [00:00<?, ?it/s]

[['O', 'H', 'H', 'O', 'H', 'H'], ['O', 'H', 'H', 'O', 'H', 'H'], ['O', 'H', 'H', 'O', 'H', 'H'], ['O', 'H', 'H', 'O', 'H', 'H'], ['O', 'H', 'H', 'O', 'H', 'H'], ['O', 'H', 'H', 'O', 'H', 'H'], ['O', 'H', 'H', 'O', 'H', 'H'], ['O', 'H', 'H', 'O', 'H', 'H']]
Time elapsed: 0.017144381999969482 s
